<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Youtube-DNN" data-toc-modified-id="Youtube-DNN-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Youtube DNN</a></span><ul class="toc-item"><li><span><a href="#召回阶段" data-toc-modified-id="召回阶段-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>召回阶段</a></span></li><li><span><a href="#排序阶段" data-toc-modified-id="排序阶段-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>排序阶段</a></span></li></ul></li></ul></div>

## Youtube DNN

> **推荐阅读**⭐️⭐️⭐️⭐️⭐️
> 1. [深入浅出地理解Youtube DNN推荐模型](https://zhuanlan.zhihu.com/p/405907646)
> 2. [YouTube深度学习推荐系统的十大工程问题](https://zhuanlan.zhihu.com/p/52504407)
> 3. [揭开YouTube深度推荐系统模型Serving之谜](https://zhuanlan.zhihu.com/p/61827629)
> 4. paper: 《Deep Neural Networks for YouTube Recommendations》

Q1：文中把推荐问题转换成多分类问题，在预测next watch的场景下，每一个备选video都会是一个分类，因此总共的分类有数百万之巨，这在使用softmax训练时无疑是低效的，这个问题YouTube是如何解决的？

    带权随机负采样


Q2：Youtube的用户对新视频有偏好，那么在模型构建的过程中如何引入这个feature？

    为了拟合用户对fresh content的bias，模型引入了“Example Age”这个feature，文中其实并没有精确的定义什么是example age。按照文章的说法猜测的话，会直接把sample log距离当前的时间作为example age。比如24小时前的日志，这个example age就是24。在做模型serving的时候，不管使用那个video，会直接把这个feature设成0。

    之所以不用视频产生的时间距当前训练时刻的间隔，可能是考虑到该特征的分布太分散了（包含几年前的视频）。这里还对时间特征进行了开方平方，增强模型的非线性能力。

    预测时置为0保证预测时处于训练的最后一刻，论文也说了还可以稍微负一些，虽然还没试过，但可以理解为处于训练窗口以后的时间去预测。

Q3：在对训练集的预处理过程中，YouTube没有采用原始的用户日志，而是对每个用户提取等数量的训练样本，这是为什么？

    减少高度活跃用户对于loss的过度影响

Q4：在确定优化目标的时候，YouTube为什么不采用经典的CTR，或者播放率（Play Rate），而是采用了每次曝光预期播放时间（expected watch time per impression）作为优化目标？

    这个问题从模型角度出发，是因为 watch time更能反应用户的真实兴趣，从商业模型角度出发，因为watch time越长，YouTube获得的广告收益越多。而且增加用户的watch time也更符合一个视频网站的长期利益和用户粘性。objective的设定应该是一个算法模型的根本性问题。

Q5：在进行video embedding的时候，为什么要直接把大量长尾的video直接用0向量代替？

    这又是一次工程和算法的trade-off，把大量长尾的video截断掉，主要还是为了节省online serving中宝贵的内存资源。当然从模型角度讲，低频video的embedding的准确性不佳是另一个“截断掉也不那么可惜”的理由。

    还可以用HashBucket去映射，对于大规模稀疏ID类特征，实际使用上用Hash不会对结果产生太大影响，反而在增量更新的情况上可能会比置为0更好。


Q6：针对某些特征，比如#previous impressions，为什么要进行开方和平方处理后，当作三个特征输入模型？

    这是很简单有效的工程经验，引入了特征的非线性。从YouTube这篇文章的效果反馈来看，提升了其模型的离线准确度。


### 召回阶段

<img src="https://yilonghao-picgo.oss-cn-hangzhou.aliyuncs.com/image-20221231090206726.png" alt="image-20221231090206726" style="zoom: 33%; display: block; margin: auto;" />

输入层是：
1. 用户观看视频序列的embedding mean pooling
2. 搜索词的embedding mean pooling
3. 地理位置embedding
4. example age
5. 性别等特征

然后是三层激活函数为ReLU的全连接层，得到用户向量，最后经过softmax层，得到每个视频的观看概率。

<img src="https://yilonghao-picgo.oss-cn-hangzhou.aliyuncs.com/image-20221231091534111.png" alt="image-20221231091534111" style="zoom:50%; display: block; margin: auto;" />

### 排序阶段

<img src="https://yilonghao-picgo.oss-cn-hangzhou.aliyuncs.com/image-20221231090319382.png" alt="image-20221231090319382" style="zoom:33%; display: block; margin: auto;" />

排序阶段的模型结构和召回阶段基本一致，主要不同点在于特征和目标。在特征方面则加入了更多细粒度的特征。

目标方面：广告是YouTube的主要盈利方式，如果仅仅使用CTR预估作为目标任务，会倾向于推荐标题党视频给用户，我们期望可以增加用户观看视频的时长，这样才能在视频中插入广告。具体而言，在训练时采用加权的损失函数，权重与样本的观看时长相关，在线上打分时，使用近似期望观看时长作为打分。